# 06 - 판다스 데이터프레임 변경

## 5.데이터프레임 변경
- 데이터 전처리를 위해 꼭 익혀두어야 할 내용입니다.
- 반복 학습과 실습을 통해서 익숙해져야 합니다.

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

* Attrition 데이터 불러오기

**[Attrition 데이터 셋 정보]**

|	구분	|	변수 명	|	내용	|	type	|	비고	|
|	----	|	----	|	----	|	----	|	----	|
|	**Target**	|	**Attrition**	|	이직여부, Yes , No	|	범주	| 1- 이직, 0- 잔류		|
|	feature	|	Age	|	나이	|	숫자	|		|
|	feature	|	DistanceFromHome	|	집-직장 거리	|	숫자	|	마일	|
|	feature	|	EmployNumber	|	사번	|	숫자	| 	|
|	feature	|	Gender	|	성별	|	범주	| Male, Female		|
|	feature	|	JobSatisfaction	|	직무 만족도	|	범주	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	feature	|	MaritalStatus	|	결혼상태	|	범주	| Single, Married, Divorced		|
|	feature	|	MonthlyIncome	|	월급	|	숫자	| 달러	|
|	feature	|	OverTime	|	야근여부	|	범주	|	Yes, No	|
|	feature	|	PercentSalaryHike	|	전년대비 급여인상율	|	숫자	|	%	|
|	feature	|	TotalWorkingYears	|	총 경력 연수	|	숫자	|		|

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition_simple2.CSV'
data = pd.read_csv(path)  

# 상위 5개 확인
data.head(5)

,Attrition,Age,DistanceFromHome,EmployeeNumber,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


In [118]:
path = 'https://bit.ly/3HaMAtZ'
data = pd.read_csv(path) 

display(data.loc[(data['Pclass'] == 2) & (data['Sex'] == 'male'), ['Age']].mean())


Age    30.876481
dtype: float64

### 5.1.열 이름 변경

- 기존 데이터프레임의 열 이름을 적절히 변경해야 할 경우가 있습니다.
- 또는 집계 결과를 가진 열 이름을 이해하기 쉽게 변경해야 할 경우도 있습니다.

**1) 일부 열 이름 변경**

- **rename()** 메소드를 사용해 변경 전후의 열 이름을 딕셔너리 형태로 나열하는 방법으로 변경합니다.
- **inplace=True** 옵션을 설정해야 변경 사항이 실제 반영이 됩니다.
- 다음과 같이 열 이름을 변경합니다.
    - DistanceFromHome → Distance
    - EmployeeNumber → EmpNo
    - JobSatisfaction → JobSat
    - MonthlyIncome → M_Income
    - PercentSalaryHike → PctSalHike
    - TotalWorkingYears → TotWY

In [3]:
# rename() 함수로 열 이름 변경
data.rename(columns={'DistanceFromHome' : 'Distance', 
                    'EmployeeNumber' : 'EmpNo',
                    'JobSatisfaction' : 'JobSat',
                    'MonthlyIncome' : 'M_Income',
                    'PercentSalaryHike' : 'PctSalHike',
                    'TotalWorkingYears' : 'TotWY'}, inplace=True)

# 확인
data.head()

,Attrition,Age,Distance,EmpNo,Gender,JobSat,MaritalStatus,M_Income,OverTime,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


**2) 모든 열 이름 변경**

- 모든 열 이름을 변경할 때는 **columns** 속성을 변경합니다.
- 변경이 필요없는 열은 기존 이름을 부여해 변경합니다. 


In [39]:
# 모든 열 이름 변경
data.columns = ['Attr','Age','Dist','EmpNo','Gen','JobSat','Marital','M_Income', 'OT', 'PctSalHike', 'TotWY']

# 확인
data.head()

ValueError: Length mismatch: Expected axis has 12 elements, new values have 11 elements

### 5.2.열 추가

- 새로운 열을 추가하여 기존 데이터에서 계산된 결괏값을 저장해야할 경우가 있습니다.
- 작년 급여액을 계산해서 열을 추가해 봅시다. 
    * 올해 급여 : MonthlyIncome(M_Income)
    * 급여 인상율 : PercentSalaryHike(PctSalHike)
    * 작년 급여 * ( 1 + PctSalHike / 100 ) = M_Income
    * 작년 급여 = M_Income / (( 1 + PctSalHike / 100 )

In [5]:
# final_amt 열 추가
data['Income_LY'] = data['M_Income'] / (1+data['PctSalHike']/100 )
data['Income_LY'] = round(data['Income_LY'])
# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0


- (참조) insert() 메소드를 사용하면 원하는 위치에 열을 추가할 수 있습니다.

    * 데이터프레임의 칼럼 위치를 너무 신경쓰지 맙시다.^^

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] 직무만족도(JobSat) 변수가 어떤 값을 몇 개 갖고 있는지 확인하세요.(.value_counts())

In [6]:
data['JobSat'].value_counts()

4    373
3    354
1    243
2    226
Name: JobSat, dtype: int64

[문2] 1,2를 불만(unsat), 3,4 를 만족(sat)으로 하는 새로운 변수JobSat2를 추가하시오.

np.where를 사용해 봅시다!

In [25]:
#jobSat2 = data['JobSat'].where(data['JobSat'] < 3, )
data['JobSat2'] = np.where(data['JobSat'].isin([1, 2]), 'unsat', 'sat')
display(data)

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Diff_Income,jobSat2,JobSat2
0,0,33,7,817,Male,3,Married,11691,No,11,14,1159.0,sat,sat
1,0,35,18,1412,Male,4,Single,9362,No,11,10,928.0,sat,sat
2,0,42,6,1911,Male,1,Married,13348,No,13,18,1536.0,unsat,unsat
3,0,46,2,1204,Female,1,Married,17048,No,23,28,3188.0,unsat,unsat
4,1,22,4,593,Male,3,Single,3894,No,16,4,537.0,sat,sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,0,32,5,1555,Female,2,Married,5878,No,12,12,630.0,unsat,unsat
1192,0,27,19,1619,Male,1,Divorced,4066,No,11,7,403.0,unsat,unsat
1193,0,29,9,1558,Male,3,Married,2451,No,18,5,374.0,sat,sat
1194,0,29,2,469,Male,3,Married,4649,No,14,4,571.0,sat,sat


[문3] 급여 인상액(Diff_Income)을 추가합시다.

In [20]:
data['Diff_Income'] = data['M_Income'] - data['Income_LY']
                                                             
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,Diff_Income,jobSat2
0,0,33,7,817,Male,3,Married,11691,No,11,14,10532.0,1159.0,sat
1,0,35,18,1412,Male,4,Single,9362,No,11,10,8434.0,928.0,sat
2,0,42,6,1911,Male,1,Married,13348,No,13,18,11812.0,1536.0,unsat
3,0,46,2,1204,Female,1,Married,17048,No,23,28,13860.0,3188.0,unsat
4,1,22,4,593,Male,3,Single,3894,No,16,4,3357.0,537.0,sat


### 5.3.열 삭제

- 뭔가를 삭제할 때는 **항상 조심x100** 해야 합니다.

**1) 열 하나 삭제**

- **drop()** 메소드를 사용해 열을 삭제합니다.
- axis=0: 행 삭제(기본 값) 
- axis=1: 열 삭제
- **inplace=True** 옵션을 지정해야 실제로 반영이 됩니다.
    * False : 삭제한 것 처럼 보여줘(조회!)


In [17]:
# 데이터프레임을 csv 파일로 저장하기
data.head()
data.to_csv('data_22020802.csv', index = False)

In [21]:
# 열 하나 삭제
data.drop('Income_LY', axis=1, inplace=True)

# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


**2) 여러 열 삭제**

- 삭제할 열을 리스트 형태로 전달해 한 번에 여러 열을 제거할 수 있습니다.

In [24]:
# 열 두 개 삭제
data.drop(['JobSat2','Diff_Income'], axis=1, inplace=True)

# 확인
data.head()

KeyError: "['JobSat2' 'Diff_Income'] not found in axis"

### 5.4.유익한 기능

- 값을 변경하거나 결괏값을 갖는 열을 추가하는 유익한 기능들을 익혀두면 좋습니다.

**1) map() 메소드**

- **map()** 메소드를 사용하면 범주형 값을 다른 값으로 쉽게 변경 할 수 있습니다.
- 다음 구문은 Gen 변수의 Male, Female을 각각 숫자 1, 0으로 변경합니다.

In [38]:
# Male -> 1, Female -> 0
data['Gen'] = data['Gen'].map({'Male': 1, 'Female': 0})

# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,Income_LY,Diff_Income,jobSat2
0,0,33,7,817,1,3,1,11691,No,10532.0,1159.0,sat
1,0,35,18,1412,1,4,0,9362,No,8434.0,928.0,sat
2,0,42,6,1911,1,1,1,13348,No,11812.0,1536.0,unsat
3,0,46,2,1204,0,1,1,17048,No,13860.0,3188.0,unsat
4,1,22,4,593,1,3,0,3894,No,3357.0,537.0,sat


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] PctSalHike,TotWY 열을 삭제하시오.

In [37]:
data.drop(['PctSalHike', 'TotWY'], axis=1, inplace=True)
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,Income_LY,Diff_Income,jobSat2
0,0,33,7,817,Male,3,1,11691,No,10532.0,1159.0,sat
1,0,35,18,1412,Male,4,0,9362,No,8434.0,928.0,sat
2,0,42,6,1911,Male,1,1,13348,No,11812.0,1536.0,unsat
3,0,46,2,1204,Female,1,1,17048,No,13860.0,3188.0,unsat
4,1,22,4,593,Male,3,0,3894,No,3357.0,537.0,sat


[문2] Marital 열의 값 Single, Married, Divorce를 각각 숫자 0,1,2로 변경하세요.(.map 사용)

In [36]:
path = 'data_22020802.csv'
data = pd.read_csv(path)

data['Marital'] = data['Marital'].map({'Single' : 0, 'Married' : 1, 'Divorced' : 2})
data.head(20)

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,PctSalHike,TotWY,Income_LY,Diff_Income,jobSat2
0,0,33,7,817,Male,3,1,11691,No,11,14,10532.0,1159.0,sat
1,0,35,18,1412,Male,4,0,9362,No,11,10,8434.0,928.0,sat
2,0,42,6,1911,Male,1,1,13348,No,13,18,11812.0,1536.0,unsat
3,0,46,2,1204,Female,1,1,17048,No,23,28,13860.0,3188.0,unsat
4,1,22,4,593,Male,3,0,3894,No,16,4,3357.0,537.0,sat
5,0,24,21,1551,Male,1,2,2296,No,14,2,2014.0,282.0,unsat
6,0,34,8,2068,Male,3,1,4404,No,12,6,3932.0,472.0,sat
7,0,30,20,1084,Male,1,1,9957,No,15,7,8658.0,1299.0,unsat
8,0,26,6,686,Female,4,1,2659,Yes,13,3,2353.0,306.0,sat
9,1,24,3,720,Female,3,0,4577,No,14,4,4015.0,562.0,sat


**2) cut() 함수**

- **크기**를 기준으로 범위를 나누어 등급을 지정하고 싶을 때 **cut() 함수**를 사용하면 좋습니다.
- 균등구간분할 : 범위 개수를 지정하면 자동으로 크기를 기준으로 나눕니다.

In [40]:
pd.cut(data['M_Income'], 3)

0        (7339.0, 13669.0]
1        (7339.0, 13669.0]
2        (7339.0, 13669.0]
3       (13669.0, 19999.0]
4         (990.01, 7339.0]
               ...        
1191      (990.01, 7339.0]
1192      (990.01, 7339.0]
1193      (990.01, 7339.0]
1194      (990.01, 7339.0]
1195    (13669.0, 19999.0]
Name: M_Income, Length: 1196, dtype: category
Categories (3, interval[float64, right]): [(990.01, 7339.0] < (7339.0, 13669.0] < (13669.0, 19999.0]]

In [41]:
# 범위를 3등분해서 등급 구하기
data['M_Income_Group'] = pd.cut(data['M_Income'], 3, labels=['a', 'b', 'c'] )

# 확인
data.head()

,Attr,Age,Dist,EmpNo,Gen,JobSat,Marital,M_Income,OT,Income_LY,Diff_Income,jobSat2,M_Income_Group
0,0,33,7,817,1,3,1,11691,No,10532.0,1159.0,sat,b
1,0,35,18,1412,1,4,0,9362,No,8434.0,928.0,sat,b
2,0,42,6,1911,1,1,1,13348,No,11812.0,1536.0,unsat,b
3,0,46,2,1204,0,1,1,17048,No,13860.0,3188.0,unsat,c
4,1,22,4,593,1,3,0,3894,No,3357.0,537.0,sat,a


In [42]:
# 범위 확인
data.groupby('M_Income_Group', as_index = False)['M_Income'].agg(['min', 'max'])

,min,max
M_Income_Group,,
a,1009,7336
b,7351,13610
c,13695,19999



- 다음 구문은 아래 범위에 따른 값을 갖는 M_Income_Group2 열을 추가합니다.
  - 10000 이하 → 'a'
  - 10000 초과 15000 이하 → 'b'
  - 15000 초과 → 'c'
- 음의 무한대는 -np.inf, 양의 무한대는 np.inf로 지정합니다.

In [43]:
# 등급 구하기
bin = [0, 10000, 15000, np.inf]
data['M_Income_Group2'] = pd.cut(data['M_Income'], bins=bin, labels=['a', 'b', 'c'])

# 범위 확인
data.groupby('M_Income_Group2', as_index = False)['M_Income'].agg(['min', 'max'])

,min,max
M_Income_Group2,,
a,1009,9998
b,10048,14852
c,15202,19999


### 5.5 종합실습

1) pandas 라이브러리를 불러오세요.

In [83]:
# 라이브러리 불러오기
import pandas as pd

2) 다음 경로의 파일을 읽어 titanic 데이터프레임을 만드세요.

- 파일 'https://bit.ly/TitanicFile'

**[titanic 데이터 셋 정보]**

- Survived: 생존여부(1=Yes, 0=No)
- Pclass: 객실등급(1=1st, 2=2nd, 3=3rd)
- Sex: 성별(male, female)
- Age: 나이
- Sibsp: 탑승한 형제자매, 배우자 수
- Parch: 탑승한 자녀, 부모 수
- Ticket: 티켓 번호
- Fare: 요금
- Cabin: 객실 번호
- Embarked: 출발한 항구((C=Cherbourg, Q=Queenstown, S=Southampton)

In [84]:
# 파일 읽어오기
path = 'https://bit.ly/TitanicFile'
titanic = pd.read_csv(path)

3) 상위 5개 행을 확인하세요.

In [85]:
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


4) 다음 요구사항에 맞게 하나의 코드셀에 순서대로 구문을 작성하고 확인하세요.

- 4-1) PassengerId, Name, Ticket, Cabin 열을 한 번에 삭제하세요.
- 4-2) Sex 열 이름을 Male로 변경하세요.
- 4-3) Male 열 값을 'male'은 1, 'female'은 0으로 변경하세요.
- 4-4) SibSp 열과 Parch 열의 값을 더한 결과를 갖는 Family 열을 추가하세요.
- 4-5) SibSp, Parch 두 열을 삭제하세요.
- 4-6) 결과를 확인하세요.

In [86]:
# 4-1) PassengerId, Name, Ticket, Cabin 열 삭제
col_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin'] # drop 리스트를 따로 지정해서 하기도함
titanic.drop(col_drop, axis=1, inplace=True)
display(titanic.head(5))

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [87]:
# 4-2) 열 이름 변경: Sex --> Male
titanic.rename(columns={'Sex' : 'Male'}, inplace=True)
display(titanic.head(5))

,Survived,Pclass,Male,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [88]:
# 4-3) Male 열 값 변경: male --> 1, female --> 0
titanic['Male'] = titanic['Male'].map({'male' : 1, 'female' : 0})
display(titanic.head(5))

,Survived,Pclass,Male,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [89]:
# 4-4) Family = SibSp + Parch
titanic['Family'] = titanic['SibSp'] + titanic['Parch']
display(titanic.head(5))

,Survived,Pclass,Male,Age,SibSp,Parch,Fare,Embarked,Family
0,0,3,1,22.0,1,0,7.2500,S,1
1,1,1,0,38.0,1,0,71.2833,C,1
2,1,3,0,26.0,0,0,7.9250,S,0
3,1,1,0,35.0,1,0,53.1000,S,1
4,0,3,1,35.0,0,0,8.0500,S,0


In [90]:
# 4-5) SibSp, Parch 열 삭제
titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)
display(titanic.head(5))

,Survived,Pclass,Male,Age,Fare,Embarked,Family
0,0,3,1,22.0,7.2500,S,1
1,1,1,0,38.0,71.2833,C,1
2,1,3,0,26.0,7.9250,S,0
3,1,1,0,35.0,53.1000,S,1
4,0,3,1,35.0,8.0500,S,0


In [93]:
# 4-6) 확인
display(titanic.head(5))

,Survived,Pclass,Male,Age,Fare,Embarked,Family
0,0,3,1,22.0,7.2500,S,1
1,1,1,0,38.0,71.2833,C,1
2,1,3,0,26.0,7.9250,S,0
3,1,1,0,35.0,53.1000,S,1
4,0,3,1,35.0,8.0500,S,0


5) 나이를 연령대로 구분하고자 합니다. 아래 조건의 값을 갖도록 변수를 추가하세요.
* 0<= <10 : '유아'
* 10<= <20 : 10대
* 20<= <30 : 20대
* 30<= <40 : 30대
* 40<= <50 : 40대
* 50<= <60 : 50대
* 60<= <70 : 60대
* 70<= <80 : 70대
* 80<=     : 80대

> 값의 범위 중 오른쪽(큰값)이 아닌 왼쪽(작은값)이 포함되도록 하려면 pd.cut 함수 옵션에 right = False 라고 지정해야 합니다.  
https://pandas.pydata.org/docs/reference/api/pandas.cut.html

In [94]:
bin = range(0, 90, 10)
label = ['유아', '10대', '20대', '30대', '40대', '50대', '60대', '70대']
titanic['Age'] = pd.cut(titanic['Age'], bins=bin, right=False, labels=label)
display(titanic.head(5))

# 범위 확인
titanic.groupby('Age', as_index = False)['Age'].agg(['min', 'max'])

,Survived,Pclass,Male,Age,Fare,Embarked,Family
0,0,3,1,20대,7.2500,S,1
1,1,1,0,30대,71.2833,C,1
2,1,3,0,20대,7.9250,S,0
3,1,1,0,30대,53.1000,S,1
4,0,3,1,30대,8.0500,S,0


,min,max
Age,,
유아,유아,유아
10대,10대,10대
20대,20대,20대
30대,30대,30대
40대,40대,40대
50대,50대,50대
60대,60대,60대
70대,70대,70대


In [81]:
display(titanic[titanic['Age'] % 10 == 0].head(20))

,Survived,Pclass,Male,Age,SibSp,Parch,Fare,Embarked,Age_group
12,0,3,1,20.0,0,0,8.0500,S,20대
30,0,1,1,40.0,0,0,27.7208,C,40대
40,0,3,0,40.0,1,0,9.4750,S,40대
79,1,3,0,30.0,0,0,12.4750,S,30대
91,0,3,1,20.0,0,0,7.8542,S,20대
113,0,3,0,20.0,1,0,9.8250,S,20대
131,0,3,1,20.0,0,0,7.0500,S,20대
157,0,3,1,30.0,0,0,8.0500,S,30대
161,1,2,0,40.0,0,0,15.7500,S,40대
177,0,1,0,50.0,0,0,28.7125,C,50대
